# Enrich Raw Input Data

The data pulled from the BigQuery ListenBrainz dataset is pretty sparse and contains only the artist's name, the user's name, and the corresponding number of listen that user has for that artist. In order to train a prediction algorithm, we must enrich the raw input data by leveraging third party data providers to provide descriptors of the artist's music which may inform user preference.

We use the [Last FM API](http://last.fm/api/) to get the data for each artist. In our code below we assume that the first matching artist is the correct one.

Our <font color="darkblue"><strong>working hypothesis</strong></font> is that if the user in question does not like any musicians that are tagged as "rock" artists, then they probably won't like The Beatles.

In [1]:
import requests
import pandas as pd

In [2]:
lastfm_api_keys = dict([x.split('\t') for x in open('lastfm.conf').read().split('\n')])

In [3]:
artists_df = pd.read_feather("input_data/artist_df.feather")

For each artist in our `artists_df` dataframe, we perform the following steps:

1. Search Last FM for the artist name
2. Grab the id for that artist (mbid)
3. Call getInfo to get more data on the artist
4. Get the tags
5. Add the tags to the columns and set as True: meaning we saw that tag for that artist



In [4]:
def get_artist_tags(artist_name: str) -> list:
    """For a provided musical artist, searches Last FM for corresponding 'tag' data, then returns that data"""
    
    tags = []
    
    search_response = requests.get(
        url=lastfm_api_keys['API Root'],
        params=dict(
            method="artist.search",
            artist=artist_name,
            api_key=lastfm_api_keys['API key'],
            format="json"
        )
    ).json()
    
    mbid = search_response['results']['artistmatches']['artist'][0]['mbid']  # unique identifier for a particular artist
    
    tag_response = requests.get(
        url=lastfm_api_keys['API Root'],
        params=dict(
            method="artist.getInfo",
            artist=artist_name,
            mbid=mbid,
            api_key=lastfm_api_keys['API key'],
            format="json"
        )
    ).json()
    
    tags = ["tag_{}".format(x['name'].lower().replace("-", " ")) for x in tag_response['artist']['tags']['tag']]
    
    return tags

In [5]:
get_artist_tags("The White Stripes")

['tag_rock',
 'tag_alternative',
 'tag_alternative rock',
 'tag_indie',
 'tag_indie rock']

Now that we've defined our function to get the data we need, let's iterate over our dataset and retrieve tag information for all of our artists.

The code block below takes a while to run, and for no good reason that I can see.

Let's do this instead - let's 

In [6]:
artists_df['artist_name'].head()

0      Radiohead
1    The Beatles
2     Pink Floyd
3      Daft Punk
4           Muse
Name: artist_name, dtype: object

Running the code below results in a couple of issues:
1. The LastFM API obviously has some sort of rate limiting, and some artists are skipped
2. Performance for iterating over dataframes is slow
3. We receive an additional PerformanceWarning for the DataFrame being highly fragmented, a result of calling `frame.insert` too many times

In [7]:
for index, row in artists_df.iterrows():
    artist_name = row['artist_name']
    
    try:
        tags = get_artist_tags(artist_name)
        
        for tag in tags:
            artists_df.at[index, tag] = True
            
    except Exception as e:
        print(e)

HTTPConnectionPool(host='ws.audioscrobbler.com', port=80): Max retries exceeded with url: /2.0/?method=artist.search&artist=Massive+Attack&api_key=74eef7c44309c5e7e9f35c1a783e774a&format=json (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f00b473bdd0>: Failed to establish a new connection: [Errno 101] Network is unreachable'))
HTTPConnectionPool(host='ws.audioscrobbler.com', port=80): Max retries exceeded with url: /2.0/?method=artist.search&artist=Bob+Dylan&api_key=74eef7c44309c5e7e9f35c1a783e774a&format=json (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f00b46d4650>: Failed to establish a new connection: [Errno 101] Network is unreachable'))


/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1684: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.obj[key] = infer_fill_value(value)


HTTPConnectionPool(host='ws.audioscrobbler.com', port=80): Max retries exceeded with url: /2.0/?method=artist.search&artist=Hans+Zimmer&api_key=74eef7c44309c5e7e9f35c1a783e774a&format=json (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f00b4655d90>: Failed to establish a new connection: [Errno 101] Network is unreachable'))
HTTPConnectionPool(host='ws.audioscrobbler.com', port=80): Max retries exceeded with url: /2.0/?method=artist.search&artist=Alice+in+Chains&api_key=74eef7c44309c5e7e9f35c1a783e774a&format=json (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f00b4655690>: Failed to establish a new connection: [Errno 101] Network is unreachable'))
HTTPConnectionPool(host='ws.audioscrobbler.com', port=80): Max retries exceeded with url: /2.0/?method=artist.search&artist=Coheed+and+Cambria&api_key=74eef7c44309c5e7e9f35c1a783e774a&format=json (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f00b4

After retrieving the tag data from LastFM API we then fill any missing information with a default of 'False'.

In [8]:
artists_df.fillna(False, inplace=True)
artists_df.head(5)

,artist_name,tag_alternative,tag_alternative rock,tag_rock,tag_indie,tag_electronic,tag_classic rock,tag_british,tag_60s,tag_pop,...,tag_jay z,tag_shoegazer,tag_hair metal,tag_rapcore,tag_underground hip hop,tag_symphonic black metal,tag_darkwave,tag_world,tag_latin,tag_spanish
0,Radiohead,True,True,True,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,The Beatles,False,False,True,False,False,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
2,Pink Floyd,False,False,True,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,Daft Punk,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Muse,True,True,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


Finally we output the enriched data to a portable file format.

In [9]:
artists_df.to_feather("enriched_data/band_tags.feather")